In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
df =pd.read_csv('/content/gdrive/My Drive/data_mi.csv',delimiter ="|" ,error_bad_lines = False, header = None)

In [0]:
import numpy as np
df.columns = ['query', 'passage', 'label_','label']
df_ones = df[df['label'] == 1 ]
# select the rows where labels are zeros but only a few 
df_zeros = df[df['label'] == 0].sample(frac=0.3)
df_new = pd.concat([df_ones,df_zeros],ignore_index=True)
x1=  np.array(df_new['query'])
x2=  np.array(df_new['passage'])
y_train1 = np.array(df_new['label_'])
y_train2 = np.array(df_new['label'])

In [0]:
y_train = np.column_stack((y_train1, y_train2))

In [7]:
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

1.12.0


Using TensorFlow backend.


In [8]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded https://tfhub.dev/google/elmo/2, Total size: 357.40MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


In [0]:
def ElmoEmbedding(x):
    return elmo_model(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['elmo']

In [0]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

In [11]:
from keras.models import Model
from keras.layers import Concatenate, Input, Dense,Dropout, LSTM, Bidirectional, Reshape
import numpy as np

# First model
first_input = Input((1, ),dtype="string")
embedding1 = Lambda(ElmoEmbedding, output_shape=(12, 1024))(first_input)
#rs = np.reshape(embedding1, (146671,1, 1024))
bilsm = Bidirectional(LSTM(512, recurrent_dropout=0.2, dropout=0.2))(embedding1)
first_dense1 = Dense(4)(bilsm)

# Second model
second_input = Input((1, ),dtype="string")
embedding2 = Lambda(ElmoEmbedding, output_shape=(12, 1024))(second_input)
bilsm_ = Bidirectional(LSTM(512, recurrent_dropout=0.2, dropout=0.2))(embedding2)
second_dense = Dense(4)(bilsm_)

# Concatenate both
merged = Concatenate()([first_dense1, second_dense])
output_layer = Dense(2,activation='softmax')(merged)
model = Model(inputs=[first_input, second_input], outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics =['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
model.fit([x1,x2],
          y_train,
          batch_size= 32,
          epochs = 20)

Epoch 1/20
36833/36833 [==============================] - 1041s 28ms/step - loss: 0.5640 - acc: 0.7322
Epoch 2/20
36833/36833 [==============================] - 1031s 28ms/step - loss: 0.5293 - acc: 0.7431
Epoch 3/20
36833/36833 [==============================] - 1043s 28ms/step - loss: 0.4926 - acc: 0.7609
Epoch 4/20
36833/36833 [==============================] - 1043s 28ms/step - loss: 0.4052 - acc: 0.8128
Epoch 5/20
36833/36833 [==============================] - 1043s 28ms/step - loss: 0.2677 - acc: 0.8874
Epoch 6/20
36833/36833 [==============================] - 1041s 28ms/step - loss: 0.1659 - acc: 0.9354
Epoch 7/20
36833/36833 [==============================] - 1040s 28ms/step - loss: 0.1229 - acc: 0.9541
Epoch 8/20
36833/36833 [==============================] - 1038s 28ms/step - loss: 0.0991 - acc: 0.9654
Epoch 9/20
36833/36833 [==============================] - 1035s 28ms/step - loss: 0.0882 - acc: 0.9686
Epoch 10/20
36833/36833 [==============================] - 1035s 28ms/ste

In [0]:
import pandas as pd
evalu = pd.read_csv('/content/gdrive/My Drive/result.csv', delimiter ='|', header = None)

In [0]:
evalu.columns = ['query_id', 'query', 'passage']
x1=  np.array(evalu['query'])
x2=  np.array(evalu['passage'])

In [0]:
res = model.predict([x1,x2])

In [0]:
result = res[:,1]
qid = np.array(evalu['query_id'])

In [0]:
def TextData2(outputfile):
  c =0
  i=0
  fw = open(outputfile,"w",encoding="utf-8")
  for j in range (10417):
    fw.write( qid[c].astype(str)  + "\t" + result[i].astype(str) + "\t" +  result[i+1].astype(str) + "\t" +  result[i+2].astype(str) + "\t" +  result[i+3].astype(str) + "\t" +  result[i+4].astype(str) + "\t" +  result[i+5].astype(str) + "\t" +  result[i+6].astype(str) + "\t" +  result[i+7].astype(str) + "\t" +  result[i+8].astype(str) + "\t" +  result[i+9].astype(str) + "\n")
    c = c+10
    i = i+10

In [0]:
outputfile = '/content/gdrive/My Drive/answer3.tsv'
TextData2(outputfile)